# Ładowanie danych

In [1]:
SUF = -3
sentences = []
with open('test.txt', 'r') as f:
    for line in f:
        sentences.append(line.rstrip('\n'))

In [2]:
s_sets = set()
s_sets_suf = set()
for i in sentences:
    for j in i.split():
        s_sets.add(j)
        s_sets_suf.add(j[SUF:])
def load_Ngrams(file):
    unigrams = {}
    unigrams_suf = {}
    bigrams = {}
    bigrams_suf = {}
    with open(file, 'r') as f:
        for line in f:
            count, w1, w2 = line.split()
            count = int(count)
            if w1 in s_sets and w2 in s_sets:
                
                if w1 not in bigrams:
                    bigrams[w1] = {}
                bigrams[w1][w2] = count
                
                if w1 not in unigrams:
                    unigrams[w1] = 1
                if w2 not in unigrams:
                    unigrams[w2] = 1
                unigrams[w1] += 1
                unigrams[w2] += 1
            if w1[SUF:] in s_sets_suf and w2[SUF:] in s_sets_suf:
                
                if w1[SUF:] not in bigrams_suf:
                    bigrams_suf[w1[SUF:]] = {}
                bigrams_suf[w1[SUF:]][w2[SUF:]] = count
                
                if w1[SUF:] not in unigrams_suf:
                    unigrams_suf[w1[SUF:]] = 1
                if w2[SUF:] not in unigrams_suf:
                    unigrams_suf[w2[SUF:]] = 1
                unigrams_suf[w1[SUF:]] += 1
                unigrams_suf[w2[SUF:]] += 1
                
    return unigrams,unigrams_suf,bigrams,bigrams_suf

In [3]:
def load_supertags(file):
    d_suf_words = {}
    d_tags = {}
    d_words = {}
    with open(file, 'r') as f:
        for line in f:
            t = line.split()[1]
            w = line.split()[0]
            ws = ("^" + w)[-3:]
            d_words[w] = t
            if ws not in d_suf_words:
                d_suf_words[ws] = {}
            if t not in d_suf_words:
                d_suf_words[ws][t] = 0
            d_suf_words[ws][t] += 1
            if t not in d_tags:
                d_tags[t] = {}
            d_tags[t][w] = 1
                
    return (d_suf_words, d_tags, d_words)

In [4]:
unigrams,unigrams_suf,bigrams,bigrams_suf = load_Ngrams("poleval_2grams.txt")

In [5]:
d_suf_words, d_tags, d_words = load_supertags("supertags.txt")

In [6]:
def load_taggrams(file):
    K = 5
    t_unigrams = {}
    t_bigrams = {}
    with open(file, 'r') as f:
        for line in f:
            count = int(line.split()[0])
            w1 = line.split()[1]
            w2 = line.split()[2]
            if w1 in d_words:
                w1 = d_words[w1]
            else:
                count = 0
            if w2 in d_words:
                w2 = d_words[w2]
            else:
                count = 0
            if count >= K:
                
                if w1 not in t_bigrams:
                    t_bigrams[w1] = {}
                t_bigrams[w1][w2] = count
                
                if w1 not in t_unigrams:
                    t_unigrams[w1] = 1
                if w2 not in t_unigrams:
                    t_unigrams[w2] = 1
                t_unigrams[w1] += 1
                t_unigrams[w2] += 1
    return t_unigrams, t_bigrams

In [7]:
t_unigrams,t_bigrams = load_taggrams("poleval_2grams.txt")

In [8]:
T = 0
for t in t_unigrams:
    T += t_unigrams[t]
print(T)
N = 0
for u in unigrams:
    N += unigrams[u]
print(N)
N_s = 0
for u in unigrams_suf:
    N_s += unigrams_suf[u]
print(N_s)

9640981
33209
20708246


# INTERPOLACJA

In [9]:
CORPUS = []
sizeofcorpus = pow(10, 6)
with open("polish_corpora.txt", 'r') as f:
    for i in range(sizeofcorpus):
        line = f.readline().lower()
        CORPUS += line.split()

In [10]:
import random
def k1k2():
    bgrams = [dict(), dict()]
    ugrams = [dict(),dict()]
    w1 = CORPUS[0]
    
    for w2 in CORPUS[1:]:
        r = random.randint(0,10)
        i = 0
        if r == 5:
            i = 1
            
        if w1 not in bgrams[i]:
            bgrams[i][w1] = {}
        if w2 not in bgrams[i][w1]:
            bgrams[i][w1][w2] = 0
        bgrams[i][w1][w2] += 1

        if w1 not in ugrams[i]:
            ugrams[i][w1] = 0
        ugrams[i][w1] += 1
        w1 = w2
    
    return ugrams[0],bgrams[0],ugrams[1],bgrams[1],
    

In [11]:
K1_unigrams, K1_bigrams, K2_unigrams, K2_bigrams = k1k2()

In [12]:
def deleted_interpolation(K1,K2):
    K = 0
    for u in K1[0]:
        K += K1[0][u]
    method_3 = 0
    method_2 = 0
    method_1 = 0
    for w1 in K2[1]:
        for w2 in K2[1][w1]:
            count_2 = 0
            if w1 in K1[1] and w2 in K1[1][w1]:
                count_2 += K1[1][w1][w2] / K1[0][w1]
            count_1 = 0
            if w2 in K1[0]:
                count_1 += K1[0][w2] / K
            count_3 = 0
            if w1 in d_words and w2 in d_words:
                if d_words[w1] in t_bigrams and d_words[w2] in  t_bigrams[d_words[w1]] and w2 in unigrams:
                    count_3 += (t_bigrams[d_words[w1]][d_words[w2]] / t_unigrams[d_words[w1]]) * (unigrams[w2] / t_unigrams[d_words[w2]])
            count_4 = 0
        
               
            if count_2 > count_1:
                if count_2 > count_3:
                    method_2 += 1
                else:
                    method_3 += 1
            else:
                if count_1 > count_3:
                    method_1 += 1
                else:
                    method_3 += 1
    print(method_3)
    print(method_2)
    print(method_1)
    return method_1 / (method_1 + method_2 + method_3), method_2 / (method_1 + method_2 + method_3), method_3 / (method_1 + method_2 + method_3)

In [13]:
lambda1, lambda2, lambdat= deleted_interpolation([K1_unigrams, K1_bigrams], [K2_unigrams, K2_bigrams])
print(lambda1, lambda2, lambdat)

3812
433533
485169
0.5259204738356275 0.46994733955257045 0.004132186611802097


In [14]:
from math import log as ln
def interpolation(sentence):
    P_w = 0
    w1 = sentence[0]
    if w1 in unigrams:
        P_w += ln(unigrams[w1] / N)
        
    for w2 in sentence[1:]:
        
        if (w2 in unigrams and w1 not in bigrams) or (w2 in unigrams and w1 in bigrams and w2 not in bigrams[w1]):
            P_w += ln(lambda1 * (unigrams[w2] / N))
            
        if w2 in unigrams and w1 in bigrams and w2 in bigrams[w1]:
            if w1 in d_words and w2 in d_words:
                if d_words[w1] in t_bigrams and d_words[w2] in  t_bigrams[d_words[w1]] and w2 in unigrams:
                    P_w += ln( lambda1 * (unigrams[w2] / N) + lambda2 * (bigrams[w1][w2] / unigrams[w1])
                              + lambdat * (t_bigrams[d_words[w1]][d_words[w2]] / t_unigrams[d_words[w1]]) * (unigrams[w2] / t_unigrams[d_words[w2]]))
            else:
                if w1[SUF:] in bigrams_suf and w2[SUF:] in bigrams_suf[w1[SUF:]]:
                    P_w +=  ln(lambda1 * (unigrams[w2] / N) + lambda2 * (bigrams[w1][w2] / unigrams[w1]) +
                        0.2 * (bigrams_suf[w1[SUF:]][w2[SUF:]] / unigrams_suf[w1[SUF:]]))
                else:
                    P_w += ln(lambda1 * (unigrams[w2] / N) + lambda2 * (bigrams[w1][w2] / unigrams[w1]))
        
            
        
        w1 = w2
    return P_w
    

In [15]:
print(interpolation("nasz przedmiot to przetwarzanie języka naturalnego".split()))

-22.592090814806102


In [16]:
#lambda1 = 0.3
#lambda2 = 0.66
def check_pos(s, a):
    c = 0
    while s != list(a[c][1]):
        c += 1
        if c > 50000:
            print('ERROR')
            break
    return c + 1

In [17]:
import itertools
count = 0
sum_s = 0
for s in sentences:
    count += 1
    perm = list(itertools.permutations(s.split()))
    ans = [[interpolation(p), p] for p in perm]
    ans.sort(key = lambda p: p[0], reverse = True)
    sum_s += 1/check_pos(s.split(),ans)
    
print("AVERAGE: ", sum_s / count)

AVERAGE:  0.35863097948860023
